In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.cluster import KMeans

# 데이터 로드
df = pd.read_csv('/content/최종 데이터셋.csv')

In [ ]:
df.columns

Index(['발생원인_기타', '피해면적_합계', '화재시간', 'latitude ', 'longitude', '기온', '풍속',
       '습도', '강수량', '임목존재코드', '임종코드', '임상코드', '수종그룹코드', 'ta', 'ws', 'hm', 'pa',
       'pv', 'td', 'ts', 'dc10Tca', 'dc10LmcsCa', '시도', '시군구', '읍면동리', '피해금액'],
      dtype='object')

In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.cluster import KMeans

# 데이터 로드
df = pd.read_csv('/content/최종 데이터셋.csv')

# 컬럼명 정리 (공백 제거)
df.columns = df.columns.str.strip()

# 예측 대상 변수
TARGET = "피해면적_합계"

# 제거할 컬럼 (기온, 습도, 풍속,'시도', '시군구', '읍면동리','피해금액)
drop_cols = ['기온', '습도', '풍속','시도', '시군구', '읍면동리','피해금액']
df = df.drop(columns=[col for col in drop_cols if col in df.columns])

# 피해면적 데이터를 1D array로 변환
X_area = df[TARGET].values.reshape(-1, 1)

# KMeans 클러스터링 (3개 구간)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df['피해면적_bin'] = kmeans.fit_predict(X_area)

# 클러스터별 평균 피해면적을 기준으로 정렬하여 올바른 라벨링 적용
cluster_means = {i: np.mean(df.loc[df['피해면적_bin'] == i, TARGET]) for i in range(3)}
sorted_clusters = sorted(cluster_means, key=cluster_means.get)  # 평균값 기준 정렬

# 재할당된 라벨 매핑
cluster_mapping = {sorted_clusters[i]: i for i in range(3)}
df['피해면적_bin'] = df['피해면적_bin'].map(cluster_mapping)

# 입력 변수 선택 (목표 변수를 제외한 나머지 컬럼 + 위도, 경도 추가)
if 'latitude' in df.columns and 'longitude' in df.columns:
    X = df.drop(columns=[TARGET])  # 위도, 경도를 그대로 사용
else:
    raise ValueError("위도(latitude)와 경도(longitude) 컬럼이 존재하지 않습니다.")

y = df[TARGET]

# 로그 변환 (피해 면적이 극단적으로 크거나 작은 값이 존재할 경우)
y = np.log1p(y)  # log(1 + y) 변환

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBM 모델 학습
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 50,
    'max_depth': 10,
    'min_data_in_leaf': 20,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'verbose': -1
}

model = lgb.LGBMRegressor(**params)
model.fit(X_train, y_train)

# 예측 및 평가
preds = model.predict(X_test)
rmse = mean_squared_error(y_test, preds) ** 0.5  # squared=False 제거
mae = mean_absolute_error(y_test, preds)
r2 = r2_score(y_test, preds)

# 결과 출력
print(f"R²Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

R²Score: 0.8453
RMSE: 0.1267
MAE: 0.0912


In [ ]:
# 각 행에 제곱을 해보자
preds = (np.exp(preds))-1
y_test = (np.exp(y_test))-1


In [ ]:
# 지도에 띄우기 위해 위치 값을 다시 삽입
df = pd.read_csv('/content/최종 데이터셋.csv')

X_test = X_test.join(df[['시도', '시군구']])

In [ ]:
X_test.sample(4)

,발생원인_기타,화재시간,latitude,longitude,강수량,임목존재코드,임종코드,임상코드,수종그룹코드,ta,...,hm,pa,pv,td,ts,dc10Tca,dc10LmcsCa,피해면적_bin,시도,시군구
1820,270,4.0,35.102708,129.012917,0.0,1.0,2.0,2.0,30.0,7.7,...,44.0,1015.1,4.6,-3.8,17.2,8.0,0.0,0,경기,양평
1747,573,1.0,35.050644,127.726731,0.0,1.0,2.0,3.0,77.0,3.6,...,52.0,1019.0,4.1,-5.3,0.4,0.0,0.0,0,서울,노원
2988,793,2.0,36.384864,127.385925,0.0,1.0,2.0,1.0,15.0,26.6,...,33.0,993.8,11.4,8.9,34.4,3.0,1.0,0,경남,산청
1362,352,2.0,37.430714,126.825650,0.0,1.0,2.0,1.0,11.0,7.1,...,21.0,1009.3,2.1,-13.8,0.0,2.0,0.0,0,경북,경주


In [ ]:
pd.DataFrame(preds).to_csv("pred.csv", index=False)
pd.DataFrame(y_test).to_csv('test.csv', index=False)

pd.DataFrame(X_test).to_csv('input.csv', index=False)